In [1]:
import findspark

findspark.init(spark_home=r'D:\ProgramFiles\Spark\spark-3.2.3-bin-hadoop3.2')
findspark.find()

'D:\\ProgramFiles\\Spark\\spark-3.2.3-bin-hadoop3.2'

In [2]:
import pyspark
pyspark.__file__

'D:\\ProgramFiles\\Spark\\spark-3.2.3-bin-hadoop3.2\\python\\pyspark\\__init__.py'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df_fhvhv = spark.read.option('header','true').parquet('fhvhv_tripdata_2021-06.parquet')

In [5]:
df_fhvhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02682|2021-06-03 21:14:32|2021-06-03 21:25:50|         231|         114|      N|                B02682|
|              B02880|2021-06-23 17:40:44|2021-06-23 18:04:19|          41|         163|      N|                B02880|
|              B02510|2021-06-06 02:07:03|2021-06-06 02:24:30|          75|         250|      N|                  null|
|              B02865|2021-06-06 04:48:19|2021-06-06 05:15:14|         158|          22|      N|                B02865|
|              B02872|2021-06-16 15:31:51|2021-06-16 16:18:09|          90|          56|      N|                B02872|
|              B02883|2021-06-21 20:39:2

In [6]:
from pyspark.sql import types
from pyspark.sql import functions

In [7]:
df_fhvhv = df_fhvhv.withColumn('pickup_datetime',df_fhvhv.pickup_datetime.cast(types.TimestampType()))
df_fhvhv = df_fhvhv.withColumn('dropoff_datetime',df_fhvhv.dropoff_datetime.cast(types.TimestampType()))

In [8]:
df_fhvhv = df_fhvhv.withColumn('pickup_sec',functions.unix_timestamp(df_fhvhv.pickup_datetime))
df_fhvhv = df_fhvhv.withColumn('dropoff_sec',functions.unix_timestamp(df_fhvhv.dropoff_datetime))

In [9]:
df_fhvhv = df_fhvhv.withColumn('diff_hour',(df_fhvhv.dropoff_sec-df_fhvhv.pickup_sec)/3600)

In [10]:
df_fhvhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+-----------+-------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|pickup_sec|dropoff_sec|          diff_hour|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+-----------+-------------------+
|              B02682|2021-06-03 21:14:32|2021-06-03 21:25:50|         231|         114|      N|                B02682|1622720672| 1622721350|0.18833333333333332|
|              B02880|2021-06-23 17:40:44|2021-06-23 18:04:19|          41|         163|      N|                B02880|1624435844| 1624437259|0.39305555555555555|
|              B02510|2021-06-06 02:07:03|2021-06-06 02:24:30|          75|         250|      N|                  null|1622911023| 1622912070|0.29083333333333333|
|              B02865|

In [11]:
df_fhvhv.registerTempTable('fhvhv_table')

D:\ProgramFiles\Spark\spark-3.2.3-bin-hadoop3.2\python\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [12]:
spark.sql(
"""
SELECT 
    diff_hour
FROM
    fhvhv_table
ORDER BY
    diff_hour DESC;
"""
).show()

+------------------+
|         diff_hour|
+------------------+
|  66.8788888888889|
|25.549722222222222|
|19.980833333333333|
|18.197222222222223|
|16.466944444444444|
|14.268888888888888|
|13.909722222222221|
|             11.67|
|11.365833333333333|
|10.984444444444444|
|           10.2675|
| 9.966388888888888|
| 9.966388888888888|
| 9.637777777777778|
| 9.624444444444444|
| 9.480277777777777|
| 9.471666666666666|
| 9.402222222222223|
| 9.393611111111111|
| 9.376944444444444|
+------------------+
only showing top 20 rows

